<a href="https://colab.research.google.com/github/VridhiJ/finetune-llma4/blob/main/Copy_of_JanusFlow_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JanusFlow Demo
In this Google Colab we will be going through a simple example of a multimodal use case involving an image and a prompt. Original paper can be found here https://arxiv.org/abs/2411.07975.


## Step 1: Install Dependencies
We need to pull the code for JanusFlow from their github repository, as well as install torch for the model and requests to handle image URLs. Be sure to run this code with a GPU (use T4 for free) to enable CUDA.

In [2]:
!pip install git+https://github.com/deepseek-ai/Janus.git
!pip install diffusers[torch]
!pip install requests

  Cloning https://github.com/deepseek-ai/Janus.git to /tmp/pip-req-build-euhjm324
  Running command git clone --filter=blob:none --quiet https://github.com/deepseek-ai/Janus.git /tmp/pip-req-build-euhjm324
  Resolved https://github.com/deepseek-ai/Janus.git to commit 146668eafecabdc6dd9f36206281d01df6a96c05
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for janus: filename=janus-1.0.0-py3-none-any.whl size=81083 sha256=050910ff7df17a102c755a0303d2127192c5301d40592ff1180089620ac4c8c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-25eohth6/wheels/c8/d7/a5/38b7a052d4b8c2eaa701a28c711b0058f88ef0c432e7d6180a
Successfully built janus
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

We are updating torchvision here because there are dependency conflicts that arose with torch

In [3]:
!pip install --upgrade torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

## Step 2: Import Libraries

In [4]:
import torch
from janus.janusflow.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images
import requests
from PIL import Image
from io import BytesIO

Python version is above 3.10, patching the collections module.
Python version is above 3.10, patching the collections module.


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/image_processing_auto.py:520: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/image_processing_auto.py:520: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


## Step 3: Load Model and Processor

In [5]:
model_path = "deepseek-ai/JanusFlow-1.3B"
vl_chat_processor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer
vl_gpt = MultiModalityCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

# Use requests to convert image to correct format
def load_image_from_url(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.94k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/365 [00:00<?, ?B/s]

Some kwargs in processor config are unused and will not have any effect: image_start_tag, image_gen_tag, mask_prompt, image_end_tag, image_tag, add_special_token, ignore_id, sft_format, num_image_tokens. 
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.09G [00:00<?, ?B/s]

## Step 4: Prepare Inputs

In [6]:
# Example picture of dog
image_url = "https://images.unsplash.com/photo-1632351459705-22a52c7a3d1d"

# Task will be a simple QA demo asking what the picture is
conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>\nDescribe the image in detail.",
        "images": [image_url],
    },
    {"role": "Assistant", "content": ""},
]

# Loading image from URL
pil_images = [load_image_from_url(image_url)]

# Processing inputs
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=pil_images,
    force_batchify=True
).to(vl_gpt.device)

## Step 5: Generating Output

In [7]:
# Using GPT to generate response
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
)

# Decoding and printing answer
answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"User: {conversation[0]['content']}")
print(f"Assistant: {answer}")

User: <image_placeholder>
Describe the image in detail.
Assistant: A brown and white dog with a brown and white coat is holding a stick in its mouth, standing on a dirt ground with a blurred background.
